#1.Preprocesamiento de las noticias
En el siguiente segmento se realizara el recorte por tamaño de 500 caracteres por cada noticia para poder utilizar el modelo de fake news, es importante que según la longitud de cada noticia, sera igual a la cantidad de fragmentos generados por dicha noticia.

pd: inicialmente este notebook se corrio utilizando colab con GPU

In [1]:
#Se inicializa el notebook generando una conexión con el drive donde se encuentran los CSV
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Se instala la librería de Transformers, esta es propia de Hugging face donde se pueden importar modelos mediante un Pipeline
#esta es especializada en trabajar con modelos basados en transformers, en este caso orientado a modelos de NLP
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 163 kB 68.1 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 


Cargamos las librerias

In [3]:
#De la libreria transformers importamos la función del pipeline que permite obtener la estructura del modelo que deseamos
from transformers import pipeline
#Usamos tqdm la cual es una función para observar los tiempos de carga de los for
from tqdm import tqdm
#la libreria nltk nos permite utilizar funciones, especialmente las orientadas a NLP, en este caso se utiliza para decargar un complemento del modelo fakenews
import nltk
#La librería pandas nos permite leer, editar, crear y exportar dataframe donde tenemos alojados los datos o donde ponemos los datos obtenidos por el modelo
import pandas as pd
import numpy as np
from nltk.tokenize.punkt import PunktSentenceTokenizer

In [4]:
#Se carga la base de datos de las noticias en la variable "noticias"
noticias = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /noticias.csv')

In [5]:
#Se imprime la noticia para observar su estructura, los datos necesarios y si cuenta con algun problema
noticias.head()

,news_id,news_url_absolute,news_init_date,news_final_date,news_title,news_text_content
0,news10006,https://www.bluradio.com/economia/precio-dolar...,2022-07-30,2022-08-14,Precio dolar hoy: la cotizacion de la divisa a...,"Este martes, 2 de agosto, el dolar alcanzo un ..."
1,news10011,https://www.semana.com/economia/macroeconomia/...,2022-07-30,2022-08-14,Es cierto: El presidente Gustavo Petro quiere ...,No hay nada mas permanente que lo temporal y a...
2,news10015,https://elcomercio.pe/respuestas/que/gustavo-p...,2022-07-30,2022-08-14,Gustavo Petro: ?Que dice el informe final de s...,El equipo de empalme del gobierno del presiden...
3,news10028,https://www.lanacion.com.ar/estados-unidos/cua...,2022-07-30,2022-08-14,Cuanto cuesta y como enviar dinero de Estados ...,?Buscas opciones para el envio de dinero de Es...
4,news10029,https://www.lanacion.com.ar/estados-unidos/com...,2022-07-30,2022-08-14,Como enviar dinero de Estados Unidos a otros p...,?Buscas opciones para el envio de dinero de Es...


In [6]:
#Creamos un segundo DataFrame en blanco
noticias_frag= pd.DataFrame()
#Al nuevo Dataframe le abrimos un nuevo feature de nombre "noticias_total", en el cual guardamos la estrutura del titulo del texto, una separación y el contenido del texto 
noticias_frag["noticias_total"]= noticias["news_title"]+ " [SEP] "+ noticias["news_text_content"]
#De la misma forma agregamos los identificadores de la noticia para que cada "noticias_total" tenga asociado su id
noticias_frag["noticia_id"]= noticias["news_id"]
#Observamos el resultado
noticias_frag.head()

,noticias_total,noticia_id
0,Precio dolar hoy: la cotizacion de la divisa a...,news10006
1,Es cierto: El presidente Gustavo Petro quiere ...,news10011
2,Gustavo Petro: ?Que dice el informe final de s...,news10015
3,Cuanto cuesta y como enviar dinero de Estados ...,news10028
4,Como enviar dinero de Estados Unidos a otros p...,news10029


In [7]:
# Esto permite utilizar el tokenizador que esta en el modelo de las Fake News
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Narrativaai/fake-news-detection-spanish")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/858k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Por la base de datos usada para entrenar el algoritmo de las fake news es nesesario realizar dos pasos

1.   La ortografia es fundamental para mas informacion de esto leer el documento
2.   El algoritmo solo recibe hasta 512 tokens de su tokenizador



In [8]:
# Se realizan algunas correcciones ortograficas
noticias_corregidas_0 = []
for noticia in tqdm(noticias_frag["noticias_total"]):
  noticia = noticia.replace("ano", "año")
  noticia = noticia.replace("Ano", "Año")
  noticia = noticia.replace("sion", "sión")
  noticia = noticia.replace("cion", "ción")
  noticias_corregidas_0.append(noticia)
noticias_frag['noticias_corr'] = noticias_corregidas_0

100%|██████████| 23377/23377 [00:01<00:00, 21241.64it/s]


In [ ]:
# Eliminamos noticias en las cuales contextualmente no podemos separar las oraciones para que genere partes menores a 512 tokens, coincidencialmente 
# la mayoria de estas no importan puesto a que solo tienen, numeros o nombres, constituyen informacion de una tabla, o hacen parte de un reporte o boletin el cual consiste en
# resumen de muchas noticias
noticias_frag = noticias_frag.drop(759).reset_index(drop=True)
noticias_frag = noticias_frag.drop(1523).reset_index(drop=True)
noticias_frag = noticias_frag.drop(1548).reset_index(drop=True)
noticias_frag = noticias_frag.drop(1785).reset_index(drop=True)
noticias_frag = noticias_frag.drop(2695).reset_index(drop=True)
noticias_frag = noticias_frag.drop(3608).reset_index(drop=True)
noticias_frag = noticias_frag.drop(4313).reset_index(drop=True)
noticias_frag = noticias_frag.drop(4630).reset_index(drop=True)
noticias_frag = noticias_frag.drop(5617).reset_index(drop=True)
noticias_frag = noticias_frag.drop(5617).reset_index(drop=True)
noticias_frag = noticias_frag.drop(5727).reset_index(drop=True)
noticias_frag = noticias_frag.drop(6496).reset_index(drop=True)
noticias_frag = noticias_frag.drop(6625).reset_index(drop=True)
noticias_frag = noticias_frag.drop(7104).reset_index(drop=True)
noticias_frag = noticias_frag.drop(7229).reset_index(drop=True)
noticias_frag = noticias_frag.drop(7230).reset_index(drop=True)
noticias_frag = noticias_frag.drop(7954).reset_index(drop=True)
noticias_frag = noticias_frag.drop(8003).reset_index(drop=True)
noticias_frag = noticias_frag.drop(8131).reset_index(drop=True)
noticias_frag = noticias_frag.drop(10137).reset_index(drop=True)
noticias_frag = noticias_frag.drop(10663).reset_index(drop=True)
noticias_frag = noticias_frag.drop(10681).reset_index(drop=True)
noticias_frag = noticias_frag.drop(11135).reset_index(drop=True)
noticias_frag = noticias_frag.drop(11143).reset_index(drop=True)
noticias_frag = noticias_frag.drop(11928).reset_index(drop=True)
noticias_frag = noticias_frag.drop(11934).reset_index(drop=True)
noticias_frag = noticias_frag.drop(11946).reset_index(drop=True)
noticias_frag = noticias_frag.drop(12141).reset_index(drop=True)
noticias_frag = noticias_frag.drop(12966).reset_index(drop=True)
noticias_frag = noticias_frag.drop(14122).reset_index(drop=True)
noticias_frag = noticias_frag.drop(14124).reset_index(drop=True)
noticias_frag = noticias_frag.drop(14357).reset_index(drop=True)
noticias_frag = noticias_frag.drop(14619).reset_index(drop=True)
noticias_frag = noticias_frag.drop(14991).reset_index(drop=True)
noticias_frag = noticias_frag.drop(14997).reset_index(drop=True)
noticias_frag = noticias_frag.drop(15247).reset_index(drop=True)
noticias_frag = noticias_frag.drop(15877).reset_index(drop=True)
noticias_frag = noticias_frag.drop(16953).reset_index(drop=True)
noticias_frag = noticias_frag.drop(17073).reset_index(drop=True)
noticias_frag = noticias_frag.drop(17308).reset_index(drop=True)
noticias_frag = noticias_frag.drop(18516).reset_index(drop=True)
noticias_frag = noticias_frag.drop(19425).reset_index(drop=True)
noticias_frag = noticias_frag.drop(19717).reset_index(drop=True)
noticias_frag = noticias_frag.drop(20683).reset_index(drop=True)
noticias_frag = noticias_frag.drop(20715).reset_index(drop=True)
noticias_frag = noticias_frag.drop(20818).reset_index(drop=True)
noticias_frag = noticias_frag.drop(22510).reset_index(drop=True)
noticias_frag = noticias_frag.drop(23026).reset_index(drop=True)

In [ ]:
# dividimos las noticias en oraciones menores a 512 tokens
noticias_intervalo= pd.DataFrame({'id':pd.Series(dtype='str'),'fragmento':pd.Series(dtype='str')}) 
pst = PunktSentenceTokenizer(lang_vars=None)
for i in tqdm(range(len(noticias_frag))):
  token_noticias = pst.tokenize(noticias_frag['noticias_corr'][i])
  lista_def = []
  x = 0
  id = 0
  c = token_noticias[id]
  while x == 0 :
    tokenizer_2 = tokenizer(c)
    if len(tokenizer_2['input_ids']) < 512:
      if id+1 < len(token_noticias):
        c = c+" "+token_noticias[id+1]
        id += 1
      else:
        lista_def.append(c)
        x = 1
    else:
      c = c[:-len(token_noticias[id])-1]
      lista_def.append(c)
      c = token_noticias[id]
  textos_divi =  {'id': noticias_frag['noticia_id'][i], 'fragmento': lista_def}
  df = pd.DataFrame(textos_divi)
  noticias_intervalo = noticias_intervalo.append(df).reset_index(drop=True)

100%|██████████| 23329/23329 [07:46<00:00, 49.97it/s]


In [ ]:
noticias_intervalo.head()

,id,fragmento
0,news10006,Precio dolar hoy: la cotización de la divisa a...
1,news10011,Es cierto: El presidente Gustavo Petro quiere ...
2,news10011,"Es decir, que no es un dinero con el que se qu..."
3,news10015,Gustavo Petro: ?Que dice el informe final de s...
4,news10028,Cuanto cuesta y como enviar dinero de Estados ...


In [ ]:
#Guardamos el resultado en un csv como backup, para tener una muestra de los fragmentos
noticias_intervalo.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_news_frag_with_ID.csv', index=False)

#Filtro Fake News

In [ ]:
# Este paso se genera en caso de solo poder llegar hasta el paso anterior
noticias_intervalo=pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_news_frag_with_ID.csv')

In [ ]:
#Guardamos en una variable en strig con el nombre del modelo que nos interesa obtener del Pipeline
ckpt = "Narrativaai/fake-news-detection-spanish"
#Cargamos el modelo en una variable fakenews, de forma que establecemos la etiqueta del modelo, el nombre del modelo y el hiperparamentro device para saber si queremos o no que utilice GPU 1=no y 0=si
fakenews = pipeline("text-classification", model=ckpt, device=0)


Downloading:   0%|          | 0.00/898 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
#Descomprimimos el complemento punkt el cual es un tokenizador propio del modelo que es necesario para su ejecución
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Corremos el modelo-preentrenado con los valores en noticias_intervalo, en forma de lista, los valores fueron divididos en paquetes de 500
#porque el modelo solo soporta pasos de maximo 570 carácteres
fake=fakenews(list(noticias_intervalo["fragmento"].values))


In [ ]:
#guardamos el resultado (que es un conjunto de diccionarios) en formato df
fake_df= pd.DataFrame(fake)

In [ ]:
fake_df.head()

,label,score
0,FAKE,0.897919
1,FAKE,0.999957
2,FAKE,0.999939
3,FAKE,0.999955
4,REAL,0.998692


In [ ]:
#Asignamos los respectivos id de noticia con cada uno de los labels y scores generados del modelo
fake_df['id'] = noticias_intervalo['id']

In [ ]:
fake_df.head()

,label,score,id
0,FAKE,0.897919,news10006
1,FAKE,0.999957,news10011
2,FAKE,0.999939,news10011
3,FAKE,0.999955,news10015
4,REAL,0.998692,news10028


In [ ]:
# Al tener todavia muchos errores de ortografia aumenta el numero de falsos positivos, por ende usaremos una probabilidad de 75% para asignar el label FAKE
fake_df[fake_df["label"]=="FAKE"]["score"].describe()

count    34728.000000
mean         0.988584
std          0.053279
min          0.500065
25%          0.999155
50%          0.999852
75%          0.999966
max          0.999976
Name: score, dtype: float64

In [ ]:
fake_df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_label_interval.csv', index=False)

In [ ]:
# Asignamos el label de REAL o FAKE para la mayor probabilidad, pero teniendo en cuenta que si es FAKE solo tomaremos fake los que tengan una probabilidad 
# mayor al 75% de las probabilidades del label FAKE sienddo este valor 0.999976
labels = []
for i in tqdm(fake_df['id'].unique()):
  df2 = fake_df[fake_df['id']== i].reset_index()
  real =[]
  fake = []
  for j in range(len(df2)):
    if df2['label'][j] == 'REAL':
      real.append(df2['score'][j])
      fake.append(1-df2['score'][j])
    else:
      fake.append(df2['score'][j])
      real.append(1-df2['score'][j])
  if np.mean(real) > np.mean(fake): 
    labels.append('REAL')
  elif np.mean(fake) > 0.999976:
    labels.append('FAKE')
  else:
    labels.append('REAL')

100%|██████████| 23329/23329 [01:37<00:00, 240.29it/s]


In [ ]:
#en esta funcion promediamos las probabilidades de los diferentes fragmentos de texto con el fin de poder encontrar si es mas probable que sea Real o Fake
labels = []
for i in tqdm(fake_df['id'].unique()):
  df2 = fake_df[fake_df['id']== i].reset_index()
  real =[]
  fake = []
  for j in range(len(df2)):
    if df2['label'][j] == 'REAL':
      real.append(df2['score'][j])
      fake.append(1-df2['score'][j])
    else:
      fake.append(df2['score'][j])
      real.append(1-df2['score'][j])
  if np.mean(real) > np.mean(fake): 
    labels.append('REAL')
  else:
    labels.append('FAKE')

100%|██████████| 23329/23329 [01:30<00:00, 258.04it/s]


In [ ]:
#convertimos lo anterior en un dataframe en el cual vamos a tener el label definitivo del detector de fake news para cada noticia
d = {'id':fake_df['id'].unique(),'Label':labels}
df = pd.DataFrame(d)
df.tail()

,id,Label
23324,news99983,REAL
23325,news99989,REAL
23326,news99991,REAL
23327,news99996,REAL
23328,news99997,REAL


In [ ]:
#guardamos este resultado en un csv
df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_definitivo_with_id.csv', index=False)

#Clasificador multiple

In [9]:
#Se descarga el clasificador, usando el pipeline zeroshot, se recomiendo poner el hiperparametro device=0 para utilizar GPU
classifier = pipeline("zero-shot-classification", 
                       model="Recognai/bert-base-spanish-wwm-cased-xnli", device=0)
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

Downloading:   0%|          | 0.00/834 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

#Obtener sinonimos de base actualizable

In [10]:
#Se carga el csv de sinonimos
sinonimos = pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Sinonimos_Caracteristicas  - Sinonimos_Caracteristicas.csv')

In [11]:
sinonimos

,Caracteristica,sinonimo
0,sostenibilidad,sostenibilidad
1,sostenibilidad,viabilidad
2,sostenibilidad,factibilidad
3,sostenibilidad,sustentabilidad
4,sostenibilidad,permanencia
...,...,...
108,otros,violencia
109,otros,literatura
110,otros,religión
111,otros,secuestro


In [12]:
#Se guardan los sinonimos en variables separadas con formado dataframe  
Sos=sinonimos[(sinonimos.Caracteristica=="sostenibilidad")]
Ali=sinonimos[(sinonimos.Caracteristica=="alianza")]
Rep=sinonimos[(sinonimos.Caracteristica=="reputación")]
Reg=sinonimos[(sinonimos.Caracteristica=="regulación")]
Inn=sinonimos[(sinonimos.Caracteristica=="innovación")]
Mac=sinonimos[(sinonimos.Caracteristica=="macroeconomía")]
Otr=sinonimos[(sinonimos.Caracteristica=="otros")]

In [13]:
#Los df son convertidos en vectores y almacenados en variables para ser utilizados en el modelo
sos=Sos['sinonimo'].to_numpy()
ali=Ali['sinonimo'].to_numpy()
rep=Rep['sinonimo'].to_numpy()
reg=Reg['sinonimo'].to_numpy()
inn=Inn['sinonimo'].to_numpy()
mac=Mac['sinonimo'].to_numpy()
otr=Otr['sinonimo'].to_numpy()

In [66]:
#Se agrupan los vectores de sinonimos en una sola variable candidate_labels
candidate_labels = [sos, ali, rep, reg, inn, mac, otr]
#Se genera la clasificación guardando los resultados en la variable Pred_L de forma que lo obtenido son dic con dos features labels y score
Pred_L = classifier(list(noticias_frag['noticias_corr'].values), candidate_labels, multi_labels=True)

In [67]:
noticias_frag.head()

,noticias_total,noticia_id,noticias_corr
0,Precio dolar hoy: la cotizacion de la divisa a...,news10006,Precio dolar hoy: la cotización de la divisa a...
1,Es cierto: El presidente Gustavo Petro quiere ...,news10011,Es cierto: El presidente Gustavo Petro quiere ...
2,Gustavo Petro: ?Que dice el informe final de s...,news10015,Gustavo Petro: ?Que dice el informe final de s...
3,Cuanto cuesta y como enviar dinero de Estados ...,news10028,Cuanto cuesta y como enviar dinero de Estados ...
4,Como enviar dinero de Estados Unidos a otros p...,news10029,Como enviar dinero de Estados Unidos a otros p...


In [68]:
# se convierte el resultado en un csv
label_df = pd.DataFrame(Pred_L)

In [69]:
#se asigna el id de la noticia a cada resultado
label_df['id'] = noticias_frag['noticia_id']

In [70]:
label_df.head()

,sequence,labels,scores,id
0,Precio dolar hoy: la cotización de la divisa a...,"[[reputación, prestigio, fama, estatus, notori...","[0.30296555161476135, 0.21928028762340546, 0.1...",news10006
1,Es cierto: El presidente Gustavo Petro quiere ...,"[[reglamentación, organización, regularización...","[0.2925281822681427, 0.2588120996952057, 0.168...",news10011
2,Gustavo Petro: ?Que dice el informe final de s...,"[[reglamentación, organización, regularización...","[0.4224588871002197, 0.1709466427564621, 0.146...",news10015
3,Cuanto cuesta y como enviar dinero de Estados ...,"[[reglamentación, organización, regularización...","[0.29498717188835144, 0.23135842382907867, 0.2...",news10028
4,Como enviar dinero de Estados Unidos a otros p...,"[[reglamentación, organización, regularización...","[0.2932043969631195, 0.20868907868862152, 0.19...",news10029


In [71]:
# en esta celda se comprueba cual es el vector de sinonimos que posee la probabilidad mas alta con el fin de poder asignar las categorias 
#(la probabilidad mas alta siempre es el primer arreglo de scores y labels)
# commo vimos anteriormente es poosible encontrar multiples noticias en un solo texto y esto lo hacce descartable por lo cual si la probabilidad de una clase no es mayor al 25%
# puede deberse a la presencia de multiples clases (lo que lo hace descartable)
scores = [x[0] for x in label_df["scores"]]
categoria =[]
for i in range(len(label_df)):
  if scores[i] > 0.25:
    if np.array_equal(label_df['labels'][i][0],sos):
      categoria.append("sostenibilidad")
    if np.array_equal(label_df['labels'][i][0],ali):
      categoria.append("alianza")
    if np.array_equal(label_df['labels'][i][0],rep):
      categoria.append("reputación")
    if np.array_equal(label_df['labels'][i][0],reg):
      categoria.append("regulación")
    if np.array_equal(label_df['labels'][i][0],inn):
      categoria.append("innovación")
    if np.array_equal(label_df['labels'][i][0],mac):
      categoria.append("macroeconomía")
    if np.array_equal(label_df['labels'][i][0],otr):
      categoria.append("otros")
  else:
    categoria.append("descartable")

In [72]:
#Agregamos la categoria al df
label_df['categoria'] = categoria
label_df.head()

,sequence,labels,scores,id,categoria
0,Precio dolar hoy: la cotización de la divisa a...,"[[reputación, prestigio, fama, estatus, notori...","[0.30296555161476135, 0.21928028762340546, 0.1...",news10006,reputación
1,Es cierto: El presidente Gustavo Petro quiere ...,"[[reglamentación, organización, regularización...","[0.2925281822681427, 0.2588120996952057, 0.168...",news10011,regulación
2,Gustavo Petro: ?Que dice el informe final de s...,"[[reglamentación, organización, regularización...","[0.4224588871002197, 0.1709466427564621, 0.146...",news10015,regulación
3,Cuanto cuesta y como enviar dinero de Estados ...,"[[reglamentación, organización, regularización...","[0.29498717188835144, 0.23135842382907867, 0.2...",news10028,regulación
4,Como enviar dinero de Estados Unidos a otros p...,"[[reglamentación, organización, regularización...","[0.2932043969631195, 0.20868907868862152, 0.19...",news10029,regulación


In [73]:
#guardamos el resultado
label_df.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorias.csv',index= False )

#Proceso para filtrar la categoria final
Teniendo los resultados filtrados por noticia tanto de fake news como de el zero-shot para la categorizacion solo debemos comprobar si es o no una fake news para agregarlo en la categoria descarable o asignar la categoria correspondiente

In [74]:
df= pd.read_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /Fake_definitivo_with_id.csv')

In [75]:
# se unen los dos dataframes por su id
df_definitive = df.merge(label_df,on='id')
df_definitive.head()

,id,Label,sequence,labels,scores,categoria
0,news10006,REAL,Precio dolar hoy: la cotización de la divisa a...,"[[reputación, prestigio, fama, estatus, notori...","[0.30296555161476135, 0.21928028762340546, 0.1...",reputación
1,news10011,REAL,Es cierto: El presidente Gustavo Petro quiere ...,"[[reglamentación, organización, regularización...","[0.2925281822681427, 0.2588120996952057, 0.168...",regulación
2,news10015,REAL,Gustavo Petro: ?Que dice el informe final de s...,"[[reglamentación, organización, regularización...","[0.4224588871002197, 0.1709466427564621, 0.146...",regulación
3,news10028,REAL,Cuanto cuesta y como enviar dinero de Estados ...,"[[reglamentación, organización, regularización...","[0.29498717188835144, 0.23135842382907867, 0.2...",regulación
4,news10029,REAL,Como enviar dinero de Estados Unidos a otros p...,"[[reglamentación, organización, regularización...","[0.2932043969631195, 0.20868907868862152, 0.19...",regulación


In [76]:
#se compruba si la noticia es falsa o se asigna la categoria de no serlo
true_cat =[]
for i in range(len(df_definitive)):
  if df_definitive['Label'][i] == 'FAKE':
    true_cat.append('descartable')
  else:
    true_cat.append(df_definitive['categoria'][i])

In [77]:
# se agrega la categoria final al dataframe
df_definitive['True_cat'] = true_cat

De aqui en adelante se realizan ciertas modificaciona al dataframe con el fin de cumplir con el formato del archivo categorizacion.csv de los entregables

In [78]:
df_definitive.rename(columns = {'id':'news_id'}, inplace = True)

In [79]:
df_definitive.head()

,news_id,Label,sequence,labels,scores,categoria,True_cat
0,news10006,REAL,Precio dolar hoy: la cotización de la divisa a...,"[[reputación, prestigio, fama, estatus, notori...","[0.30296555161476135, 0.21928028762340546, 0.1...",reputación,reputación
1,news10011,REAL,Es cierto: El presidente Gustavo Petro quiere ...,"[[reglamentación, organización, regularización...","[0.2925281822681427, 0.2588120996952057, 0.168...",regulación,regulación
2,news10015,REAL,Gustavo Petro: ?Que dice el informe final de s...,"[[reglamentación, organización, regularización...","[0.4224588871002197, 0.1709466427564621, 0.146...",regulación,regulación
3,news10028,REAL,Cuanto cuesta y como enviar dinero de Estados ...,"[[reglamentación, organización, regularización...","[0.29498717188835144, 0.23135842382907867, 0.2...",regulación,regulación
4,news10029,REAL,Como enviar dinero de Estados Unidos a otros p...,"[[reglamentación, organización, regularización...","[0.2932043969631195, 0.20868907868862152, 0.19...",regulación,regulación


In [80]:
df_definitive['nombre_equipo']=  ['Goofy_Goobers' for x in range(len(df_definitive))]
df_definitive.head()

,news_id,Label,sequence,labels,scores,categoria,True_cat,nombre_equipo
0,news10006,REAL,Precio dolar hoy: la cotización de la divisa a...,"[[reputación, prestigio, fama, estatus, notori...","[0.30296555161476135, 0.21928028762340546, 0.1...",reputación,reputación,Goofy_Goobers
1,news10011,REAL,Es cierto: El presidente Gustavo Petro quiere ...,"[[reglamentación, organización, regularización...","[0.2925281822681427, 0.2588120996952057, 0.168...",regulación,regulación,Goofy_Goobers
2,news10015,REAL,Gustavo Petro: ?Que dice el informe final de s...,"[[reglamentación, organización, regularización...","[0.4224588871002197, 0.1709466427564621, 0.146...",regulación,regulación,Goofy_Goobers
3,news10028,REAL,Cuanto cuesta y como enviar dinero de Estados ...,"[[reglamentación, organización, regularización...","[0.29498717188835144, 0.23135842382907867, 0.2...",regulación,regulación,Goofy_Goobers
4,news10029,REAL,Como enviar dinero de Estados Unidos a otros p...,"[[reglamentación, organización, regularización...","[0.2932043969631195, 0.20868907868862152, 0.19...",regulación,regulación,Goofy_Goobers


In [81]:
df_definitive = df_definitive.drop(columns=['Label','labels','scores','categoria'])

In [82]:
df_definitive.head()

,news_id,sequence,True_cat,nombre_equipo
0,news10006,Precio dolar hoy: la cotización de la divisa a...,reputación,Goofy_Goobers
1,news10011,Es cierto: El presidente Gustavo Petro quiere ...,regulación,Goofy_Goobers
2,news10015,Gustavo Petro: ?Que dice el informe final de s...,regulación,Goofy_Goobers
3,news10028,Cuanto cuesta y como enviar dinero de Estados ...,regulación,Goofy_Goobers
4,news10029,Como enviar dinero de Estados Unidos a otros p...,regulación,Goofy_Goobers


In [83]:
df_definitive.rename(columns = {'True_cat':'categoria'}, inplace = True)

In [84]:
df_definitive.to_csv('/content/drive/MyDrive/Carpeta Datatón/Documentos /categorizacion_inicial.csv',index= False )